<hr style="color:green" />
<h1 style="color:green">COSC2673 Assignment 2: Image Classification for Cancerous Cells</h1>
<h2 style="color:green">File 21: PyTorch First CNN model test on Main data, using Custom Config</h2>
<hr style="color:green" />

<p>
The AlexNet implementation, taken directly across, did not perform well. Drilling into what it was actually doing, it's clear that it's designed for larger sized images, and that many layers with those configurations was likely stripping way too much information
</p>
<p>
Create a custom, simple CNN structure appropriate to the 27x27 pixel image size
</p>

In [ ]:
import pandas as pd
import numpy as np
import os
import cv2

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image


Configure this script as to whether it runs on Google Colab, or locally

In [ ]:
# When on Google Colab, running full training, change both to true. Locally, advised set both to false
isGoogleColab = False
useFullData = False

In [ ]:
# In local, the base directory is the current directory
baseDirectory = "./"

if isGoogleColab:
    from google.colab import drive
    
    # If this is running on Google colab, assume the notebook runs in a "COSC2673" folder, which also contains the data files 
    # in a subfolder called "image_classification_data"
    drive.mount("/content/drive")
    !ls /content/drive/'My Drive'/COSC2673/

    # Import the directory so that custom python libraries can be imported
    import sys
    sys.path.append("/content/drive/MyDrive/COSC2673/")

    # Set the base directory to the Google Drive specific folder
    baseDirectory = "/content/drive/MyDrive/COSC2673/"

Import the custom python files that contain reusable code

In [ ]:
import data_basic_utility as dbutil
import graphing_utility as graphutil
import statistics_utility as statsutil

import a2_utility as a2util
import pytorch_utility as ptutil
from pytorch_utility import CancerBinaryDataset
from pytorch_utility import CancerCellTypeDataset


# randomSeed = dbutil.get_random_seed()
randomSeed = 266305
print("Random Seed: " + str(randomSeed))

In [ ]:
# this file should have previously been created in the root directory
dfImages = pd.read_csv(baseDirectory + "images_main.csv")

In [ ]:
# Get The training Split and the Validation Split
dfImagesTrain = dfImages[dfImages["trainValTest"] == 0].reset_index()
dfImagesVal = dfImages[dfImages["trainValTest"] == 1].reset_index()
dfImagesTest = dfImages[dfImages["trainValTest"] == 2].reset_index()

print(dfImagesTrain.shape)
print(dfImagesVal.shape)
print(dfImagesTest.shape)

dfImagesTrain.head()

Note: The definition of the Custom Datasets for both the isCancerous data and the Cell Type data are defined in the pytorch_utility.py file.

Also, rather than loading all the training images and calculating the mean and standard deviation values in here, that was run separately in file 05a.PyTorchGetMeanAndStd.ipynb

Here we can just define the values to use, which shouldn't change unless the data is reloaded and a new train/validation/test split is generated

In [ ]:
train_mean, train_std = ptutil.getTrainMeanAndStdTensors()
print(train_mean)
print(train_std)

In [ ]:
# Create a tranform operation that also normalizes the images according to the mean and standard deviations of the images
transform_normalize = transforms.Compose(
    [transforms.ToPILImage(),
    transforms.ToTensor(), 
    transforms.Normalize(train_mean, train_std)])


In [ ]:
cancerous_training_data = None

# Create a custom Dataset for the training and validation data
if useFullData:
    cancerous_training_data = CancerBinaryDataset(isGoogleColab, dfImagesTrain, baseDirectory, transform=transform_normalize)
else:
    # For testing in a small dataset
    dfImagesTrainTest = dfImagesTrain.iloc[range(1000), :].reset_index()
    cancerous_training_data = CancerBinaryDataset(isGoogleColab, dfImagesTrainTest, baseDirectory, transform=transform_normalize, target_transform=None)

cancerous_validation_data = CancerBinaryDataset(isGoogleColab, dfImagesVal, baseDirectory, transform=transform_normalize, target_transform=None)
cancerous_test_data = CancerBinaryDataset(isGoogleColab, dfImagesTest, baseDirectory, transform=transform_normalize, target_transform=None)

# Create data loaders
cancerous_train_dataloader = DataLoader(cancerous_training_data, batch_size=32, shuffle=True, num_workers=2)
cancerous_val_dataloader = DataLoader(cancerous_validation_data, batch_size=32, shuffle=True, num_workers=2)
cancerous_test_dataloader = DataLoader(cancerous_test_data, batch_size=32, shuffle=True, num_workers=2)

Now, create a class for the basic, Fully Connected Neural Network. For this basic NN, we will use 3 fully connected layers. The number of features in this will be 27 x 27 x 3, or 2187.

Layer 1: Input is the images, which are 27 x 27 pixels, with 3 color values (RGB). Experiment initially with 1458 nodes
Layer 2: Input is 1458 from the the previous layer, down to 729
Layer 3: Input is 729 from the the previous layer, since this is a binary classification problem, the output will be 2 classes

In this, we will use the **ReLU** Activation Function. This is the Rectified Linear Unit function, which allows the function to become non-linear

In [ ]:
# Create a class for the Neural Network
class PT_CNN_IsCancerous(nn.Module):

    # In the constructor, initialize the layers to use
    def __init__(self):
        super(PT_CNN_IsCancerous, self).__init__()

        # first, define the subsampling methods. Though they are used multiple times, these are the
        # operations, so only need to be defined once
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        # define the Activation methods to use
        self.relu = nn.ReLU(inplace=True)
        self.sigmoid = nn.Sigmoid()

        # define the convolution layers

        # input should be 27x27x3. Apply a 3x3 filter, therefore, output should be 25x25x32 (channels aka feature maps)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1)
        # There will be a Relu
        # Then a MaxPool of 2x2, halving the dimensions per feature map
        # So input is 12x12x32. Apply a 3x3 filter, also include padding=1, as this is already quite small, and lets consider the edges
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)    
        # There will be a Relu        
        # So input is 12x12x64. Apply a 3x3 filter, also include padding=1, as this is already quite small, and lets consider the edges
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # Then a MaxPool of 2x2, halving the dimensions per feature map
        # Then an Average Pool of keeping the dimensions as 6x6
        
        # define the fully connected neural layers
        self.fc1 = nn.Linear(128 * 6 * 6, 4608)
        self.fc2 = nn.Linear(4608, 2306)
        self.fc3 = nn.Linear(2306, 2)

    # Create the forward function, which is used in training
    def forward(self, x):

        # print("Init Shape: " + str(x.shape))

        # Process the first 2 convolution layers, applying maxpooling
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)

        # Then process the remaining convolution layers without any pooling
        x = self.relu(self.conv2(x))        
        x = self.relu(self.conv3(x))

        # Then apply a max pool and average pool on the result
        x = self.maxpool(x)
        #x = self.avgpool(x)

        # Flatten: This should convert to tensors that are acceptable for the input into the NN 3 layers
        x = x.view(x.size(0), 128 * 6 * 6)

        # Now process the 3 layers of the Fully Connected NN
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))        
        # x = self.relu(self.fc3(x))        
        # x = self.fc3(x)        

        # return the result
        return x


Now train the Convolutional Neural Network Model.

During training, we will use the following:
- Softmax Cross Entropy Loss as our Loss function. This is a good Loss function that basically converts scores for each class into probabilities
- The Adam Optimizer, which is a version of Gradient Descent
- Initially, just 10 epochs

In [ ]:
# set the Learning Rate to use
learning_rate = 0.0001
epochsToUse = 10

net = PT_CNN_IsCancerous()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(epochsToUse):
    print("Starting Epoch " + str(epoch) + "...")
    for i, data in enumerate(cancerous_train_dataloader, 0):
        # Get the inputs
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Perform Forward and Backward propagation then optimize the weights
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

Training Time in Nelson's Local Environment on the full data takes a very long time, stopped after 100 minutes. This will need to be done in Colab.

First, Predict on the training data so that we can find the training error.

In [ ]:
correct, total = 0,  0
predictions = []

# Set the Neural Network into evaluation (test) mode
net.eval()

step=0

y_train_cancerous = []
y_train_pred_cancerous = []

# Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
showStep = False
for i, data in enumerate(cancerous_train_dataloader, 0):
    inputs, labels = data

    outputs = net(inputs)
    class_score, predicted = torch.max(outputs.data, 1)
    
    # Loop through the batch, build the lists of the raw label and prediction values
    for j in range(len(labels)):
        y_train_cancerous.append(labels[j].item())
        y_train_pred_cancerous.append(predicted[j].item())

        if showStep:
            print(class_score[j])

    showStep=False

    predictions.append(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


print("Evaluate the Training Predictions and Error: \n")

print('Confusion matrix: \n')
print(confusion_matrix(y_train_cancerous, y_train_pred_cancerous))

print("\n- Accuracy Score: " + str(accuracy_score(y_train_cancerous, y_train_pred_cancerous)))
print("- Precision Score: " + str(precision_score(y_train_cancerous, y_train_pred_cancerous)))
print("- Recall Score: " + str(recall_score(y_train_cancerous, y_train_pred_cancerous)))
print("- F1 Score: " + str(f1_score(y_train_cancerous, y_train_pred_cancerous)))

In [ ]:
a2util.getClassificationROC("IsCancerous", "Training", y_train_cancerous, y_train_pred_cancerous)

Now Predict according to the Validation data and evaluate. While looping through here, we will need to get out the Labels from the data loader, because the order of predictions in the batches do not match the order of the original Target values in the dataset (because we turned Shuffle on)

In [ ]:
correct, total = 0,  0
predictions = []

# Set the Neural Network into evaluation (test) mode
net.eval()

step=0

y_val_cancerous = []
y_val_pred_cancerous = []

# Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
for i, data in enumerate(cancerous_val_dataloader, 0):
    inputs, labels = data

    outputs = net(inputs)
    class_score, predicted = torch.max(outputs.data, 1)
    
    # Loop through the batch, build the lists of the raw label and prediction values
    for j in range(len(labels)):
        y_val_cancerous.append(labels[j].item())
        y_val_pred_cancerous.append(predicted[j].item())

    predictions.append(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()



print("Evaluate the Validation Predictions and Error: \n")

print('Confusion matrix: \n')
print(confusion_matrix(y_val_cancerous, y_val_pred_cancerous))

print("\n- Accuracy Score: " + str(accuracy_score(y_val_cancerous, y_val_pred_cancerous)))
print("- Precision Score: " + str(precision_score(y_val_cancerous, y_val_pred_cancerous)))
print("- Recall Score: " + str(recall_score(y_val_cancerous, y_val_pred_cancerous)))
print("- F1 Score: " + str(f1_score(y_val_cancerous, y_val_pred_cancerous)))

In [ ]:
a2util.getClassificationROC("IsCancerous", "Validation", y_val_cancerous, y_val_pred_cancerous, 2)

Now Predict and evaluate on the test Set

In [ ]:
correct, total = 0,  0
predictions = []

# Set the Neural Network into evaluation (test) mode
net.eval()

step=0

y_test_cancerous = []
y_test_pred_cancerous = []

# Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
for i, data in enumerate(cancerous_test_dataloader, 0):
    inputs, labels = data

    # This should convert the image tensors into vectors
    inputs = inputs.view(-1, 27 * 27 * 3)

    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    
    # Loop through the batch, build the lists of the raw label and prediction values
    for j in range(len(labels)):
        y_test_cancerous.append(labels[j].item())
        y_test_pred_cancerous.append(predicted[j].item())

    predictions.append(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


print("Evaluate the Test Predictions and Error: \n")

print('Confusion matrix: \n')
print(confusion_matrix(y_test_cancerous, y_test_pred_cancerous))

print("\n- Accuracy Score: " + str(accuracy_score(y_test_cancerous, y_test_pred_cancerous)))
print("- Precision Score: " + str(precision_score(y_test_cancerous, y_test_pred_cancerous)))
print("- Recall Score: " + str(recall_score(y_test_cancerous, y_test_pred_cancerous)))
print("- F1 Score: " + str(f1_score(y_test_cancerous, y_test_pred_cancerous)))

In [ ]:
a2util.getClassificationROC("IsCancerous", "Test", y_test_cancerous, y_test_pred_cancerous, 2)

# Google Colab Results
<hr />

Binary with Sigmoid as Final Activation Function
- **Training**
- Accuracy Score: 0.9518948577261708
- Precision Score: 0.9426820475847152
- Recall Score: 0.9230497705612425
- F1 Score: 0.9327626181558766
- **Validation**
- Accuracy Score: 0.8894277400581959
- Precision Score: 0.9178981937602627
- Recall Score: 0.8972712680577849
- F1 Score: 0.9074675324675324

Binary with ReLU as Final Activation Function
- **Training**
- **Validation**


Binary with no Final Activation Function
- **Training**
- **Validation**


**Results from 3 Layer FC-NN with Dropout**
- Training F1: 0.923854447439353
- Validation F1: 0.897025171624714

Binary Sigmoid performs maybe 1 percentage point better than the NN


Now also train a model for CellType Predictions

In [ ]:
celltype_training_data = None

# Create a custom Dataset for the training and validation data
if useFullData:
    celltype_training_data = CancerCellTypeDataset(isGoogleColab, dfImagesTrain, baseDirectory, transform=transform_normalize)
else:
    # For testing in a small dataset
    dfImagesTrainTest = dfImagesTrain.iloc[range(1000), :].reset_index()
    celltype_training_data = CancerCellTypeDataset(isGoogleColab, dfImagesTrainTest, baseDirectory, transform=transform_normalize, target_transform=None)

celltype_validation_data = CancerCellTypeDataset(isGoogleColab, dfImagesVal, baseDirectory, transform=transform_normalize)
celltype_test_data = CancerCellTypeDataset(isGoogleColab, dfImagesTest, baseDirectory, transform=transform_normalize)

# Create data loaders
celltype_train_dataloader = DataLoader(celltype_training_data, batch_size=32, shuffle=True, num_workers=2)
celltype_val_dataloader = DataLoader(celltype_validation_data, batch_size=32, shuffle=True, num_workers=2)
celltype_test_dataloader = DataLoader(celltype_test_data, batch_size=32, shuffle=True, num_workers=2)

Create a class for the Cell Type Neural Network model. The structure of the class will be fundamentally the same, only the model will need to output 4 classes

In [ ]:
# Create a class for the Neural Network
class PT_CNN_CellType(nn.Module):

    # In the constructor, initialize the layers to use
    def __init__(self):
        super(PT_CNN_CellType, self).__init__()


        # first, define the subsampling methods. Though they are used multiple times, these are the
        # operations, so only need to be defined once
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        # define the Activation methods to use
        self.relu = nn.ReLU(inplace=True)
        self.softmax = nn.Softmax(dim=1)

        # define the convolution layers

        # input should be 27x27x3. Apply a 3x3 filter, therefore, output should be 25x25x32 (channels aka feature maps)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1)
        # There will be a Relu
        # Then a MaxPool of 2x2, halving the dimensions per feature map
        # So input is 12x12x32. Apply a 3x3 filter, also include padding=1, as this is already quite small, and lets consider the edges
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)    
        # There will be a Relu        
        # So input is 12x12x64. Apply a 3x3 filter, also include padding=1, as this is already quite small, and lets consider the edges
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # Then a MaxPool of 2x2, halving the dimensions per feature map
        # Then an Average Pool of keeping the dimensions as 6x6
        
        # define the fully connected neural layers
        self.fc1 = nn.Linear(128 * 6 * 6, 4608)
        self.fc2 = nn.Linear(4608, 2306)
        self.fc3 = nn.Linear(2306, 4)

    # Create the forward function, which is used in training
    def forward(self, x):

        # print("Init Shape: " + str(x.shape))

        # Process the first 2 convolution layers, applying maxpooling
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)

        # Then process the remaining convolution layers without any pooling
        x = self.relu(self.conv2(x))        
        x = self.relu(self.conv3(x))

        # Then apply a max pool and average pool on the result
        x = self.maxpool(x)
        #x = self.avgpool(x)

        # Flatten: This should convert to tensors that are acceptable for the input into the NN 3 layers
        x = x.view(x.size(0), 128 * 6 * 6)

        # Now process the 3 layers of the Fully Connected NN
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))        
        # x = self.fc3(x)
        # x = self.relu(self.fc3(x))
        x = F.softmax(self.fc3(x), dim=1)        

        # return the result
        return x



Now train the Fully Connected Neural Network Model. Use the same configuration (objective function, optimizer etc) as the Binary Classifier

In [ ]:
# set the Learning Rate to use
learning_rate = 0.0001
epochsToUse = 10

net = PT_CNN_CellType()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

for epoch in range(epochsToUse):
    print("Starting Epoch " + str(epoch) + "...")
    for i, data in enumerate(celltype_train_dataloader, 0):
        # Get the inputs
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Perform Forward and Backward propagation then optimize the weights
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

Predict on the Training Set to get the Training Accuracy and Error

In [ ]:
correct, total = 0,  0
predictions = []

# Set the Neural Network into evaluation (test) mode
net.eval()

y_train_celltype = []
y_train_pred_celltype = []
y_train_pred_celltype_scores = []

showBatch=True

# Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
for i, data in enumerate(celltype_train_dataloader, 0):
    inputs, labels = data

    outputs = net(inputs)

    # outputs.data contains a tensor of size 4 for each record, with a score for each class. Use max
    # to select the class of the highest score for the prediction.
    class_score, predicted = torch.max(outputs.data, 1)
    
    # Loop through the batch, build the lists of the raw label and prediction values
    for j in range(len(labels)):
        y_train_celltype.append(labels[j].item())
        y_train_pred_celltype.append(predicted[j].item())
        y_train_pred_celltype_scores.append(outputs.data[j].tolist())

    showBatch = False
    
    predictions.append(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


print("Evaluate the Training Predictions and Error: \n")

print('Confusion matrix: \n')
print(confusion_matrix(y_train_celltype, y_train_pred_celltype))

print("\n- Accuracy Score: " + str(accuracy_score(y_train_celltype, y_train_pred_celltype)))
print("- Precision Score: " + str(precision_score(y_train_celltype, y_train_pred_celltype, average="micro")))
print("- Recall Score: " + str(recall_score(y_train_celltype, y_train_pred_celltype, average="micro")))
print("- F1 Score: " + str(f1_score(y_train_celltype, y_train_pred_celltype, average="micro")))

In [ ]:
for i in range(5):
    print(y_train_pred_celltype_scores[i])

In [ ]:
a2util.getClassificationROC("CellType", "Training", y_train_celltype, y_train_pred_celltype, 4, y_train_pred_celltype_scores)

Predict on the Validation data and evaluate the results

In [ ]:
correct, total = 0,  0
predictions = []

# Set the Neural Network into evaluation (test) mode
net.eval()

step=0

y_val_celltype = []
y_val_pred_celltype = []
y_val_pred_celltype_scores = []

# Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
for i, data in enumerate(celltype_val_dataloader, 0):
    inputs, labels = data

    outputs = net(inputs)
    class_score, predicted = torch.max(outputs.data, 1)
    
    # Loop through the batch, build the lists of the raw label and prediction values
    for j in range(len(labels)):
        y_val_celltype.append(labels[j].item())
        y_val_pred_celltype.append(predicted[j].item())
        y_val_pred_celltype_scores.append(outputs.data[j].tolist())

    predictions.append(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


accuracy = (correct/total) * 100
print("Evaluate the Validation Predictions and Error: \n")

print('Confusion matrix: \n')
print(confusion_matrix(y_val_celltype, y_val_pred_celltype))

print("\n- Accuracy Score: " + str(accuracy_score(y_val_celltype, y_val_pred_celltype)))
print("- Precision Score: " + str(precision_score(y_val_celltype, y_val_pred_celltype, average="micro")))
print("- Recall Score: " + str(recall_score(y_val_celltype, y_val_pred_celltype, average="micro")))
print("- F1 Score: " + str(f1_score(y_val_celltype, y_val_pred_celltype, average="micro")))

In [ ]:
a2util.getClassificationROC("CellType", "Validation", y_val_celltype, y_val_pred_celltype, 4, y_val_pred_celltype_scores)

Now Predict and Evaluate on the Test Set

In [ ]:
correct, total = 0,  0
predictions = []

# Set the Neural Network into evaluation (test) mode
net.eval()

y_test_celltype = []
y_test_pred_celltype = []
y_test_pred_celltype_scores = []

showBatch=True

# Looping through this dataloader essentially processes them in batches of 32 (or whatever the batchsize is configured in the data loader
for i, data in enumerate(celltype_test_dataloader, 0):
    inputs, labels = data

    # This should convert the image tensors into vectors
    inputs = inputs.view(-1, 27 * 27 * 3)

    outputs = net(inputs)

    # outputs.data contains a tensor of size 4 for each record, with a score for each class. Use max
    # to select the class of the highest score for the prediction.
    class_score, predicted = torch.max(outputs.data, 1)
    
    # Loop through the batch, build the lists of the raw label and prediction values
    for j in range(len(labels)):
        y_test_celltype.append(labels[j].item())
        y_test_pred_celltype.append(predicted[j].item())
        y_test_pred_celltype_scores.append(outputs.data[j].tolist())

    showBatch = False
    
    predictions.append(predicted)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()


print("Evaluate the Test Predictions and Error: \n")

print('Confusion matrix: \n')
print(confusion_matrix(y_test_celltype, y_test_pred_celltype))

print("\n- Accuracy Score: " + str(accuracy_score(y_test_celltype, y_test_pred_celltype)))
print("- Precision Score: " + str(precision_score(y_test_celltype, y_test_pred_celltype, average="micro")))
print("- Recall Score: " + str(recall_score(y_test_celltype, y_test_pred_celltype, average="micro")))
print("- F1 Score: " + str(f1_score(y_test_celltype, y_test_pred_celltype, average="micro")))

In [ ]:
a2util.getClassificationROC("CellType", "Test", y_test_celltype, y_test_pred_celltype, 4, y_test_pred_celltype_scores)

# Google Colab Results
<hr />

Multiclass with Softmax as Final Activation Function
- **Training**
- Accuracy Score: 0.842414189102973
- Precision Score: 0.842414189102973
- Recall Score: 0.842414189102973
- F1 Score: 0.842414189102973
- **Validation**
- Accuracy Score: 0.7827352085354026
- Precision Score: 0.7827352085354026
- Recall Score: 0.7827352085354026
- F1 Score: 0.7827352085354026


Multiclass with ReLU as Final Activation Function
- **Training**
- **Validation**


Multiclass with no Final Activation Function
- **Training**
- Accuracy Score: 0.9679724384330739
- Precision Score: 0.9679724384330739
- Recall Score: 0.9679724384330739
- F1 Score: 0.9679724384330739
- **Validation**
- Accuracy Score: 0.7575169738118331
- Precision Score: 0.7575169738118331
- Recall Score: 0.7575169738118331
- F1 Score: 0.7575169738118331

**Results from 3 Layer FC-NN with Dropout**
- Training F1: 0.8442005869592957
- Validation F1: 0.7778855480116392

<h1>Analysis of Performance and Accuracy</h1>

<strong>Binary Classification - IsCancerous</strong>
<p>
In this model we see that the model has a very low training error and a high accuracy value, in this experiment achieving a 97.3% accuracy. This indicates that the model has <strong>low bias</strong>. However, when predicting on the validation dataset, the validation error rises. It can be seen that the Accuracy for the validation predictions is 84.4%, and the are under the ROC Curve is significantly less, with a value of 0.824. This is an indication of <strong>high variance</strong>, and in combination with low bias, indicates a possible problem of <strong>overfitting</strong>
</p>

<strong>Multi-class Classification - Cell Type</strong>
<p>
Similarly, the Cell Type model has a very low training error and a high accuracy value of 99.5%, indicating that the model has <strong>low bias</strong>. However, the Cell Type model has a considerably worse accuracy of only 76.5% when predicting on the validation dataset. A ROC curve is generated for each class, and it can be seen that the model performs better at predicting some classes compared to others. The average the under the ROC Curve score, with a value of 0.838. This indication of <strong>high variance</strong>, and in combination with low bias, indicates a possible problem of <strong>overfitting</strong>.
</p>